In [3]:
from bs4 import BeautifulSoup as bs
import time
import sqlite3 
import re
from datetime import datetime
from selenium import webdriver
import json
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os





conn = sqlite3.connect('db.sqlite3')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS news
        (company_id text, dtime datetime, news text, news_url text UNIQUE)''')



# Check updates for specific Firms

In [67]:
dictio = {}
dictio['ПАО Сбербанк'] = 3043
name = dictio.keys()





def search_by_companies(*companies):
    for company in companies:
#         create the dict to control the number of new events
        company_pre = {company:0}
        request = urlopen('https://www.e-disclosure.ru/Event/Page?companyId=' + str(dictio[company]) +'&year=2020')
        soup = bs(request, 'lxml')

        
        # It is supposed to infinitely parse ---> don't need to check more than 20 first values
        result = soup.find_all('td')[:15]

        if len(result) > company_pre[company]:
            i = 1
            for td in range ((len(result) - result_pre)//3):

                    print(name, 'Дата публикации - ' + str(result[i].text),result[i+1].text,result[i+1].a.get('href'),sep = '\n')
                    i+=3
            company_pre[company] = len(result)

search_by_companies('ПАО Сбербанк')

# SQL database

# Test and training

In [50]:
conn = sqlite3.connect('Chinook_Sqlite.sqlite')

# Создаем курсор - это специальный объект который делает запросы и получает их результаты
cursor = conn.cursor()

cursor.execute("SELECT Name FROM Artist LIMIT 3")

# Получаем результат сделанного запроса
results = cursor.fetchall()
results2 =  cursor.fetchall()

print(*results)   # [('A Cor Do Som',), ('Aaron Copland & London Symphony Orchestra',), ('Aaron Goldberg',)]

# Не забываем закрыть соединение с базой данных
conn.close()

('AC/DC',) ('Accept',) ('Aerosmith',)


# Working 

In [7]:
conn = sqlite3.connect('Interfax_bot.sqlite')

cursor = conn.cursor()


cursor.execute("select Client_name from Clients")
print(cursor.fetchall())

[('alish',)]


### Adding new clients

In [97]:
conn = sqlite3.connect('Interfax_bot.sqlite')

cursor = conn.cursor()


client_chat = 779224126
client_companies = '[2314,4526,124]'

# Данные нужно передавать кортежем. Даже одну запись

cursor.execute("insert into Clients values (Null, ?, ?);", (client_chat, client_companies))

conn.commit()

#### Function to retrieve company codes as a set

In [104]:
def retrieve_code(string):
    dic = list(map(str, string.replace('{','').replace('}','').split(',')))
    dic = set(dic)
    return dic

In [105]:
print(retrieve_code('{3445,4252,2156,1345}'))

{'3445', '1345', '4252', '2156'}


#### Function to create a string from a set

In [115]:
def create_string(array):
    string = '{'
    for i in array:
        string = string + str(i) + ','
    string = string[:-1] + '}'
    return string

In [116]:
i = set([5434, 134, 670, 243])

print(create_string(i))

{5434,670,134,243}


In [94]:
clients = {779224126 : []} # a new client's chat_id is added ones he presses /start

diction ={'18278':['Ale','Ma','Joh'], '2464':['a','b','c']} # the reverse dictio containing company numbers and clients' chat_id


## Dictionary with companies - Done

In [124]:
url = 'https://www.e-disclosure.ru/poisk-po-soobshheniyam'

driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.get(url)

python_button = driver.find_element_by_id('butt') #FHSU
python_button.click() #click fhsu link

python_button2 = driver.find_element_by_id('pageSize')
python_button2.click()

soup = bs(driver.page_source, 'lxml')
res = soup.find_all('a')
# select all the relevant passages from the table. Empirically found them.
companies = {}
for a  in range(46,2445,2):
#     remove comma only when need to create a dict
#     companies[res[a].text] = res[a].get('href')[-5:]
    print(res[a].text, res[a].get('href'))

driver.quit()

for i in companies.keys():
    if companies[i].startswith('='):
        companies[i] = companies[i].replace('=','')

    if companies[i].startswith('d='):
        companies[i] = companies[i].replace('d=','')
companies

In [113]:
company = list(companies.keys())
comp_id = list(companies.values())

data = {'company':company, 'comp_id':comp_id}
df = pd.DataFrame.from_dict(data)
df

In [123]:
df.to_csv('interfax_companies.csv', encoding = 'windows - 1251')

# Telegram bot

### News check

In [3]:
def news_check(message):
    url = 'https://www.e-disclosure.ru/'
# set up driver for working with js - BS4 will not find the table 
    driver = webdriver.Firefox()
    driver.implicitly_wait(30)
    driver.get(url)
# press button 'search'
    python_button = driver.find_element_by_id('butt') #FHSU
    python_button.click() #click fhsu link
    
    soup = bs(driver.page_source, 'lxml')
    res = soup.find_all('a')

    for a in range(42,72,2):
        bot.send_message(message.chat.id, res[a].text + '\n' + str(res[a+1].text) + ' ' + str(res[a+1].get('href')))


### Subscription module

#### Finding table with last 20 news 

In [48]:
def webdriver()
    url = 'https://www.e-disclosure.ru/poisk-po-soobshheniyam'

    driver = webdriver.Firefox()
    driver.implicitly_wait(30)
    driver.get(url)

    python_button = driver.find_element_by_id('butt') #FHSU
    python_button.click() #click fhsu link
    #  Выбрать пункт все и нажать
    python_button2 = driver.find_element_by_xpath('//*[@id="pageSize"]/option[2]')
    python_button2.click()

    time.sleep(4)
    soup = bs(driver.page_source, 'lxml')
    res = soup.find_all('tbody')

    table = res[-9]

    driver.quit()


### Retrieving code

In [16]:
def firm_code_from_link(link):
    equal = link.rfind('=')
    code = link[equal+1:]
    return code

456


### Sending news to all the firms subscribers

In [ ]:
def subscribers(link,info):
    global diction
    
    code = firm_code_from_link(link)
    
    for i in diction[code]:
        bot.send_message(i,info)
        

#### Choosing only new events with the help of date-time and returning them

In [37]:
td_last = 160520201800
def send_news():

    global td_last
    
    #  Here i need to define all the variables I will use in this module
    def number_date(text):
        '''Function to convert date into an int for comparison and searching relevant news'''
        pre_res = text.split()
        one = pre_res[0].replace('.','') 
        two = pre_res[1].replace(':','')
        res = int(one+two)
        return res



    iterat = table.contents  # all tr in the table


    for child in range(0,len(iterat),2):
    # for some reason there are empty cells in [0],[2],[4] and etc. rows, so I need to go through odd ones
        date = iterat[child].contents[1].text # first td with the date and time
        if number_date(date) > td_last:
    #         iterat[child].contents[3] is the second td containing company name and the event name. However, there are two <a> tags, 
    #         one with company name, the other with the event. Therefore another method, contents[0].text - company and contents[3].text - event, is used.
            info = str(iterat[child].contents[3].contents[0].text) + '\n' + str(iterat[child].contents[3].contents[3].text) + ' ' + str(iterat[child].contents[3].contents[3].get('href'))
            subscribers(iterat[child].contents[3].contents[0].get('href'),info)
    # td_last is the last time we checked news - it is equiualent to the top row news data in the current session.

    
    td_last = number_date(iterat[0].contents[1].text) 




### The Bot

In [49]:
import telebot

token = '1145245715:AAGmd4yheC8AlBpZ5ssHqQm9-1hpt_mkAT0'

bot = telebot.TeleBot(token)

keyboard1 = telebot.types.ReplyKeyboardMarkup()
keyboard1.row('Последние новости', 'Выбрать компании')

keyboard2 = telebot.types.InlineKeyboardMarkup(); #наша клавиатура
key_one = telebot.types.InlineKeyboardButton(text='Новости', callback_data='news'); #кнопка «Да»
keyboard2.add(key_one); #добавляем кнопку в клавиатуру
key_two = telebot.types.InlineKeyboardButton(text='Выбрать компании', callback_data='companies');
keyboard2.add(key_two);



@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет, я бот раскрытия. В текущей версии я буду отправлять тебе новости о последних раскрытиях в реальном времени.')
    bot.send_message(message.chat.id, 'Какую опцию выберешь?', reply_markup = keyboard2)
    chat_id = message.chat.id
#     I need to add clients chat_id to SQLite database later
    global clients
    clients[chat_id] = []

@bot.message_handler(commands=['end'])
def end_msg(message):
    bot.send_message(message.chat.id, 'Спасибо, теперь я буду отправлять информацию по выбранным компаниям в реальном времени')
    
@bot.message_handler(content_types = ['text'])
def initiate(message):
    msg = bot.send_message(message.chat.id, 'Напиши название компании')
    bot.register_next_step_handler(msg,choice)
def choice(message):
    text = message.text
    if text == 'Сбербанк':
        bot.send_message(message.chat.id, 'Добавил')
        bot.send_message(message.chat.id, 'Если нужна еще фирма, напиши ее название. Когда выберешь все компании, напиши "/end".')
    
    
    
        

@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    res = call.message
    if call.data == "news": #call.data это callback_data, которую мы указали при объявлении кнопки
        news_check(res)
    elif call.data == "companies":
        flow_news(res)
        
        
        

bot.polling()

In [47]:
print(diction)

{'18278': ['Ale', 'Ma', 'Joh'], '2464': ['a', 'b', 'c'], 779224126: ''}


# Check daily news

In [23]:
def scraping_data():
    news_list= []
    news_adress_list = []
    company_link_list = []

    
    r = urlopen(base_domain)
    soup = bs(r, 'lxml')

    a = soup.find_all('div',class_='tab selected')
    print(a)

    
    
scraping_data()
#     for a in soup.select('[class*=live] a[href*="www.e-disclosure.ru/portal/event.aspx?"]'):
#         news_adress_list.append(a.get('href'))
#     for a in soup.select('[class*=live] a[href*="www.e-disclosure.ru/portal/company.aspx?"]'):
#         company_link_list.append(re.sub("[^0-9]","", a.get('href')[-6:]))


# don't know how to work yet        
#     for i in range(len(company_link_list)):
#     	c.execute("INSERT OR IGNORE INTO news_newsfeed (company_id, added_at, news_short, news_url) VALUES (?,?,?,?)",(company_link_list[i],datetime.now(),news_list[i],news_adress_list[i]))
#     conn.commit()
#     time.sleep(30)



[]


# Testing

In [7]:
scraping_data()

[]

In [ ]:
def inf_scraping():
    while True:
        scraping_data()

inf_scraping()

companies = []
companies2 = []


with open('emitents.csv', 'r') as file:
    for row in file:
        companies.append([row])

for i in companies:
    for b in i:
        companies2.append(b.split(';'))

companies_final = companies2[1:]

c.execute('''CREATE TABLE IF NOT EXISTS companies_companies
    (company_id INTEGER UNIQUE, ticker TEXT, company_full_name TEXT, company_short_name TEXT, official_site TEXT, ceo TEXT, industry TEXT, company_description TEXT)''')

for i in companies_final:
    data = (i[0], i[1], i[2] ,i[4] ,i[5] ,i[6] ,i[7] ,i[8])
    c.execute("INSERT INTO companies_companies (company_id,ticker,company_full_name,company_short_name,official_site,ceo,industry,company_description) VALUES (?,?,?,?,?,?,?,?)", data)
    conn.commit()

c.close()

stocks = []

with open('stocks.csv', 'r') as file:
    for row in file:
        stocks.append(row.split(';'))
stocks = stocks[1:]
for stocks_data in stocks:
    c.execute('INSERT INTO companies_share (ticker, company_id, listing_level, instrument_type, nominal, currency, issue_amount) VALUES (?,?,?,?,?,?,?)', stocks_data)
    conn.commit()
c.close()